Import library

In [ ]:
pip install tensorflow

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_dir = '/content/drive/My Drive/Capstone Project/Dataset/train'
test_dir = '/content/drive/My Drive/Capstone Project/Dataset/test'
print("Training directory:", os.listdir(train_dir))
print("Testing directory:", os.listdir(test_dir))

Mounted at /content/drive
Training directory: ['Aceh', 'Bali', 'Betawi', 'Buketan', 'Cendrawasih', 'Ceplok', 'Corak Insang', 'Dayak', 'Geblek Renteng', 'Gunungan', 'Ikat Celup', 'Jlamprang', 'Kawung', 'Lasem', 'Liong', 'Maluku', 'Madura', 'Megamendung', 'Parang', 'Poleng', 'Prada', 'Pring Sedapur', 'Sekar', 'Sidoluhur', 'Sidomukti', 'Singa Barong', 'Tambal', 'Truntum', 'Tujuh Rupa', 'Tumpal']
Testing directory: ['Aceh', 'Bali', 'Betawi', 'Buketan', 'Ceplok', 'Corak Insang', 'Cendrawasih', 'Dayak', 'Geblek Renteng', 'Gunungan', 'Ikat Celup', 'Kawung', 'Lasem', 'Liong', 'Madura', 'Maluku', 'Megamendung', 'Parang', 'Poleng', 'Prada', 'Pring Sedapur', 'Sekar', 'Sidoluhur', 'Sidomukti', 'Singa Barong', 'Tambal', 'Truntum', 'Tujuh Rupa', 'Tumpal', 'Jlamprang']


In [ ]:
BATCH_SIZE = 64

Data preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

def build_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

Found 2400 images belonging to 30 classes.
Found 600 images belonging to 30 classes.


In [ ]:
def feature_extractor(inputs):
    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model(inputs)
    return x

def classifier(inputs, num_classes):
    x = GlobalAveragePooling2D()(inputs)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    return x

In [ ]:
def define_model(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    features = feature_extractor(inputs)
    output = classifier(features, num_classes)
    model = Model(inputs=inputs, outputs=output)
    return model

model = define_model((224, 224, 3), 30)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048   23587712  
                             )                                   
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                              

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

Epoch 1/20
75/75 [==============================] - 989s 13s/step - loss: 3.1654 - accuracy: 0.1696 - val_loss: 24.9453 - val_accuracy: 0.0333
Epoch 2/20
75/75 [==============================] - 65s 868ms/step - loss: 2.5971 - accuracy: 0.2892 - val_loss: 36.1635 - val_accuracy: 0.0333
Epoch 3/20
75/75 [==============================] - 64s 852ms/step - loss: 2.2183 - accuracy: 0.3875 - val_loss: 3.8999 - val_accuracy: 0.0333
Epoch 4/20
75/75 [==============================] - 64s 852ms/step - loss: 1.9345 - accuracy: 0.4629 - val_loss: 4.3491 - val_accuracy: 0.0333
Epoch 5/20
75/75 [==============================] - 64s 850ms/step - loss: 1.7087 - accuracy: 0.5092 - val_loss: 3.8906 - val_accuracy: 0.0300
Epoch 6/20
75/75 [==============================] - 63s 841ms/step - loss: 1.4708 - accuracy: 0.5763 - val_loss: 6.2445 - val_accuracy: 0.0333
Epoch 7/20
75/75 [==============================] - 63s 834ms/step - loss: 1.3414 - accuracy: 0.6058 - val_loss: 5.0445 - val_accuracy: 0.038

In [ ]:
model.save('/content/drive/MyDrive/Capstone Project/batik_classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

19/19 [==============================] - 13s 670ms/step - loss: 2.7135 - accuracy: 0.4867
Test Loss: 2.7134501934051514
Test Accuracy: 0.4866666793823242


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'plt' is not defined

InceptionV3

In [ ]:
def feature_extractor(inputs):
    base_model = InceptionV3(weights='imagenet', include_top=False)
    x = base_model(inputs)
    return x

def classifier(inputs, num_classes):
    x = GlobalAveragePooling2D()(inputs)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    return x

In [ ]:
def define_model(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    features = feature_extractor(inputs)
    output = classifier(features, num_classes)
    model = Model(inputs=inputs, outputs=output)
    return model

model = define_model((224, 224, 3), 30)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

SyntaxError: unmatched ')' (<ipython-input-13-5922650f6e1c>, line 8)

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

In [ ]:
model.save('/content/drive/MyDrive/Capstone Project/batik_classification_model_2.h5')

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()